In [ ]:
%aiida
import urllib.parse as urlparse
import ipywidgets as ipw
from IPython.display import display, clear_output
from aiidalab_widgets_base import viewer
import nglview
import numpy as np

In [ ]:
def plot(x, y, ids, title=None, xlabel=None, ylabel=None, clr_label=None, with_links=False):
    import bokeh.plotting as bpl
    import bokeh.models as bmd
    from bokeh.io import output_notebook, show
    output_notebook(hide_banner=True)
    fig = bpl.figure(
        toolbar_location=None,
        title=title,
        x_axis_label=xlabel,
        y_axis_label=ylabel,
        tools=['tap', 'zoom_in', 'zoom_out', 'pan'], 
        output_backend='webgl',
    )
    source = bmd.ColumnDataSource(data=dict(x=x, y=y))
    
    fig.line('x', 'y', source=source)
    fig.circle('x', 'y', size=10, fill_color="white", source=source)
        
    show(fig)

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
uuid = urlparse.parse_qs(url.query)['uuid'][0]
workcalc = load_node(uuid=str(uuid))
ism = workcalc.outputs.output_parameters['isotherm']
isotherm = np.array([ism['pressure'], ism['loading_absolute_average'], ism['loading_absolute_dev']]).transpose()
p, load, calc_ids = zip(*isotherm)

# Detailed Report

## Structure

In [ ]:
display(viewer(workcalc.inputs.structure))

## Isotherm plot

In [ ]:
plot(p, load, calc_ids,
     xlabel="Pressure [{}]".format(ism['pressure_unit']),
     ylabel="Metane loading [{}]".format(ism['loading_absolute_unit']))

## Computed properties

In [ ]:
display(viewer(workcalc.outputs.output_parameters))